#**Section 2: SQL**

###Importing required libraries

In [0]:
from pyspark.sql import SparkSession, functions
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

 

##Defining files schemas 


In [0]:
#departments schema
departments_schema= StructType([\
    StructField("id",IntegerType(), False),\
    StructField("department",StringType(),True) ])

#jobs schema 
jobs_schema = StructType([\
    StructField("id",IntegerType(), False),\
    StructField("job",StringType(),True) ])

#hired_employees schema 
hired_employees_schema = StructType([\
    StructField("id",IntegerType(), False),\
    StructField("name",StringType(),True),\
    StructField("datetime",StringType(),True),\
    StructField("department_id",StringType(),True),\
    StructField("job_id",StringType(),True) ])
    


##Loading files

In [0]:
#departments file loading
departments = spark.read.format("csv")\
    .option("header", "false")\
    .schema(departments_schema)\
    .load("dbfs:/FileStore/globant_files/departments.csv")

jobs = spark.read.format("csv")\
    .option("header", "false")\
    .schema(jobs_schema)\
    .load("dbfs:/FileStore/globant_files/jobs.csv")

hired_employees = spark.read.format("csv")\
    .option("header", "false")\
    .schema(hired_employees_schema)\
    .load("dbfs:/FileStore/globant_files/hired_employees.csv")

##Creating temporal views

In [0]:
#creating temp views to work with SQL
departments.createOrReplaceTempView("departments")
jobs.createOrReplaceTempView("jobs")
hired_employees.createOrReplaceTempView("hired_employees")

##Defining queries parameters

In [0]:
year_to_analize=2021
exclude_all_nulls=False
exclude_name_nulls=False
exclude_datetime_nulls=False
exclude_department_nulls=False
exclude_job_nulls=False

In [0]:
def additional_filters(exclude_all_nulls,exclude_name_nulls,exclude_datetime_nulls,exclude_department_nulls,exclude_job_nulls):
    #exclude any null-valued fields rows
    if exclude_all_nulls:
        additional_all_filters="""
            AND h.name IS NOT NULL
            AND h.datetime IS NOT NULL
            AND h.department_id IS NOT NULL
            AND h.job_id IS NOT NULL
            """
    else:
        additional_all_filters=" "
    
    if exclude_name_nulls:
        additional_name_filters=""" AND h.name IS NOT NULL """
    else:
        additional_name_filters=" "
    
    if exclude_datetime_nulls:
        additional_datetime_filters=""" AND h.datetime IS NOT NULL """
    else:
        additional_datetime_filters=" "
    
    if exclude_department_nulls:
        additional_department_filters=""" AND h.department IS NOT NULL """
    else:
        additional_department_filters=" "
    
    if exclude_job_nulls:
        additional_job_filters=""" AND h.job IS NOT NULL """
    else:
        additional_job_filters=" "

    return additional_all_filters+additional_name_filters+additional_datetime_filters+additional_department_filters+additional_job_filters
    

##Solving first requirement

In [0]:
req_sql_1= spark.sql(f"""
    WITH base_table as (
    SELECT h.id
        ,h.name
        ,to_timestamp(h.datetime, "yyyy-MM-dd'T'HH:mm:ss'Z'") as datetime
        ,d.department
        ,j.job
    FROM hired_employees as h
        LEFT JOIN departments as d
        ON h.department_id=d.id
        LEFT JOIN jobs as j 
        ON h.job_id=j.id
        WHERE 1=1
        AND year(to_timestamp(h.datetime, "yyyy-MM-dd'T'HH:mm:ss'Z'"))={year_to_analize}
        {additional_filters(exclude_all_nulls,exclude_name_nulls,exclude_datetime_nulls,exclude_department_nulls,exclude_job_nulls)}
    ), hired_by_q as (
    SELECT 
    department
    ,job
    ,CASE WHEN month(datetime) BETWEEN 1 AND 3 THEN 'Q1' 
          WHEN month(datetime) BETWEEN 4 AND 6 THEN 'Q2' 
          WHEN month(datetime) BETWEEN 7 AND 9 THEN 'Q3'
          WHEN month(datetime) BETWEEN 10 AND 12 THEN 'Q4'
          ELSE NULL 
    END AS quarters
    ,count(DISTINCT(id)) hired    
    FROM base_table
    WHERE 1=1
    GROUP BY 
    department
    ,job
    ,CASE WHEN month(datetime) BETWEEN 1 AND 3 THEN 'Q1' 
          WHEN month(datetime) BETWEEN 4 AND 6 THEN 'Q2' 
          WHEN month(datetime) BETWEEN 7 AND 9 THEN 'Q3'
          WHEN month(datetime) BETWEEN 10 AND 12 THEN 'Q4'
          ELSE NULL 
    END
    )
    SELECT 
    department
    ,job
    ,COALESCE(Q1, 0) AS Q1
    ,COALESCE(Q2, 0) AS Q2
    ,COALESCE(Q3, 0) AS Q3
    ,COALESCE(Q4, 0) AS Q4
    FROM hired_by_q
    PIVOT (
        SUM(hired) 
        FOR quarters IN ('Q1' AS Q1, 'Q2' AS Q2, 'Q3' AS Q3, 'Q4' AS Q4))
    ORDER BY department,job
                     """)

req_sql_1.display()

department,job,Q1,Q2,Q3,Q4
null,Administrative Assistant I,0,0,1,0
null,Analog Circuit Design manager,0,0,0,1
null,Assistant Media Planner,0,0,0,1
null,Database Administrator III,0,0,0,1
null,Dental Hygienist,0,1,0,0
null,Junior Executive,0,1,0,0
null,Nurse,0,0,1,0
null,Occupational Therapist,1,0,0,0
null,Project Manager,0,0,1,0
null,Research Assistant II,0,1,0,0


In [0]:
req_sql_2= spark.sql(f"""
        WITH base_table as (
        SELECT   h.id
                ,h.name
                ,to_timestamp(h.datetime, "yyyy-MM-dd'T'HH:mm:ss'Z'") as datetime
                ,h.department_id
                ,d.department
        FROM hired_employees as h
            LEFT JOIN departments as d
            ON h.department_id=d.id
        WHERE 1=1
        AND year(to_timestamp(h.datetime, "yyyy-MM-dd'T'HH:mm:ss'Z'"))={year_to_analize}
        {additional_filters(exclude_all_nulls,exclude_name_nulls,exclude_datetime_nulls,exclude_department_nulls,exclude_job_nulls)}
        )
        , hired_by_dept as (
        SELECT department_id as id 
        ,department
        ,count(DISTINCT(id)) as hired
        FROM base_table
        WHERE 1=1
        GROUP BY department_id
        ,department
        )
        SELECT hd.id 
        ,hd.department
        ,hd.hired
        FROM hired_by_dept as hd
        GROUP BY 
        hd.id 
        ,hd.department
        ,hd.hired
        HAVING hired>(SELECT avg(hired) FROM hired_by_dept)
        ORDER BY hired DESC
                     """)

req_sql_2.display()

id,department,hired
8,Support,221
5,Engineering,208
6,Human Resources,204
7,Services,204
4,Business Development,187
3,Research and Development,151
9,Marketing,143
